In [1]:
import requests
import os
from os import path
from bs4 import BeautifulSoup

In [2]:
url = 'https://archive.org/search.php?query=title%3A%28%E6%BB%BF%E6%B4%B2%E6%97%A5%E5%A0%B1%29+AND+description%3A%28Mansh%C5%AB+Nipp%C5%8D%29&sort=date&&and[]=year%3A'
download_uri = 'https://archive.org/download/'
source_type = 'source="/services/img/'
        
pages = {
    1906 : list(range(0,7)),
    1907 : list(range(0,6)),
    1908 : list(range(0,9)),
    1927 : list(range(2,3)),
    1928 : list(range(0,9)),
    1929 : list(range(0,9)),
    1930 : list(range(0,9)),
    1931 : list(range(0,9)),
    1932 : list(range(0,9)),    
    1933 : list(range(0,9)),
    1934 : list(range(0,9)),
    1935 : list(range(0,6))
}

In [3]:
#Function that, given a year, a month and a filename, checks if the year/month folders exist and saves the file accordingly.
#When downloading a new year, the function creates both the year and month folder.
#When downloadinga new month, it creates the nested month folder. 
#Then, it saves the file in the new path (/year/month/filename) and appends the filename in the file containing all the filenames of the year.

def create_folder(year, month, filename):
    if(not(year in os.listdir())):
        os.mkdir(str(year))
        os.mkdir(str(year)+'/'+str(month))
    elif(not(month in os.listdir(str(year)))):
        os.mkdir(str(year)+'/'+str(month))
    file = open(str(year) + ".txt",'a')
    new_path = str(year)+'/'+str(month) + '/'+ filename
    os.rename(filename, new_path)

In [4]:
#Function that checks if a file is present in the corresponding year and month folder.

def check_if_downloaded(year, month, name):
    if(year in os.listdir()):
        if(month in os.listdir(str(year))):
            return name in os.listdir(str(year)+'/'+str(month))

In [5]:
#Function that creates the file containing the list of downloaded files

def create_year_file(year):
    if(not path.exists(str(year)+'.txt')):
        list_files_year = open(str(year) + ".txt",'w')
        list_files_year.close()

In [6]:
def web_scraping(url, year, page, class_type):
    new_url = url + str(year)+'&sort=date&page=' + str(page)
    req = requests.get(new_url)
    soup = BeautifulSoup(req.content, 'html.parser')
    return soup.find_all(class_ = class_type)

In [7]:
#Function that appends the new file name to the file containing the list of the already downloaded files of the year

def append_name(year, name):
    year_file = open(str(year) + ".txt",'a')
    year_file.write(name + '\n')
    year_file.close()

In [8]:
#Function that checks if a specific file has already been downloaded (therefore, its name is written in the file of the corresponding year)

def check_if_written(year, name):
    year_file = open(str(year) + ".txt",'r')
    if not(name+'\n') in year_file:
        append_name(year, name)
    else:
        year_file.close()

In [9]:
#Function that downloads and saves the file, which is then moved to its folder 

def download_file(url, file_name):
    r = requests.get(url)
    file = open(file_name + ".pdf",'wb')
    file.write(r.content)
    file.close()
    return file

In [10]:
def run_downloader(url, pages, download_uri, source_type):
    for y in pages.keys():
        create_year_file(y)
        for j in pages[y]:
            content = web_scraping(url, str(y), str(j), "item-img")
            for c in content:
                retrieved_string = str(c)
                if retrieved_string.__contains__(source_type):
                    title = retrieved_string.split(source_type)[1].split('" style=')[0]
                    name = title.split('manshu-nippo-')[1]
                    year = name.split('.')[0]
                    month = name.split('.')[1].split('.')[0]
                
                    if(check_if_downloaded(year, month, (name + ".pdf"))):
                        check_if_written(year, name)
                    else:
                        new_uri = download_uri + title + '/'+ name +'.pdf'
                        file = download_file(new_uri, name)
                        create_folder(year, month, file.name)
                        append_name(year, name)

In [11]:
run_downloader(url, pages, download_uri, source_type)

KeyboardInterrupt: 